<a href="https://colab.research.google.com/github/theindianwriter/CS6910-assignment_2/blob/main/DL_assignment_2_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np #for linear algebra operations
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image #for preprocessing the images

In [14]:
# PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

In [15]:
#enabling gpu 
device = "cpu"
if (torch.cuda.is_available()):
    device = "cuda"

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/My Drive/nature_12K.zip" -d "/content/drive/My Drive/"

In [17]:
training_folder_path = "/content/drive/MyDrive/inaturalist_12K/train"

In [18]:
#list down all the classes present in the dataset
classes = sorted([folder_name  for folder_name in os.listdir(training_folder_path) if not folder_name.startswith('.')])
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [19]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    #creating a thumbnail of the image of the given size preserving the aspect ratio
    src_image.thumbnail(size,Image.ANTIALIAS)
    #creating a background image 
    new_image = Image.new("RGB", size, bg_color)
    #pasting the src image into it 
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
    return new_image

In [34]:
new_training_folder_path = "../Natural_Dataset/train"

In [21]:
image_size = (128,128)

if os.path.exists(new_training_folder_path):
    #shutil.rmtree(new_training_folder_path)

for root,folders,_ in os.walk(training_folder_path):
    for folder in folders:
        print("resizing the images and saving for the folder ",folder)
        new_folder = os.path.join(new_training_folder_path,folder)

        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        
        file_names = os.listdir(os.path.join(root,folder))
        
        for file_name in file_names:
            if file_name.startswith('.'):
                continue
            file_path = os.path.join(root,folder,file_name)
            image = Image.open(file_path)
            resized_image = resize_image(image,image_size,"black")
            save_as = os.path.join(new_folder,file_name)
            resized_image.save(save_as)

print("resizing and saving done")

        


resizing the images and saving for the folder  Reptilia
resizing the images and saving for the folder  Mammalia
resizing the images and saving for the folder  Arachnida
resizing the images and saving for the folder  Plantae
resizing the images and saving for the folder  Aves
resizing the images and saving for the folder  Amphibia
resizing the images and saving for the folder  Insecta
resizing the images and saving for the folder  Animalia
resizing the images and saving for the folder  Mollusca
resizing the images and saving for the folder  Fungi
resizing and saving done


In [33]:
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

def load_dataset(dataset_path,data_augmentation = False,batch_size = 50):
    if data_augmentation:
        transformation = transforms.Compose([
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.3),
            transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        transformation = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=mean,std = std)                               
        ])

    full_dataset = torchvision.datasets.ImageFolder(
        root=dataset_path,
        transform=transformation
    )

    train_size = int(0.9 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    return train_loader,test_loader

In [32]:
print(device)

cuda


In [36]:
class Network(nn.Module):

    def __init__(self,num_of_classes,filters,filter_size,activation_fn,stride_len = 1,padding_len = 1,max_pool_kernel_size = 2,max_pool_stride_len = 2,dropdout_ratio = 0.0,dense_neuron_size = 20000,img_size = (128,128)):
        super(Network,self).__init__()

        self.activation_fn = activation_fn
        

        Out_dim_H = img_size[0]
        Out_dim_W = img_size[1]

        for F in filter_size:
            Out_dim_H =  int(((Out_dim_H - F + 2*padding_len)/stride_len)) + 1
            Out_dim_W =  int(((Out_dim_W - F + 2*padding_len)/stride_len)) + 1
            Out_dim_H =  int(((Out_dim_H - max_pool_kernel_size)/max_pool_stride_len))+ 1
            Out_dim_W =  int(((Out_dim_W - max_pool_kernel_size)/max_pool_stride_len)) + 1


        self.conv1 = nn.Conv2d(in_channels = 3,out_channels = filters[0],kernel_size = filter_size[0],stride = stride_len,padding = padding_len)
        self.conv2 = nn.Conv2d(in_channels = filters[0],out_channels = filters[1],kernel_size = filter_size[1],stride = stride_len,padding = padding_len)
        self.conv3 = nn.Conv2d(in_channels = filters[1],out_channels = filters[2],kernel_size = filter_size[2],stride = stride_len,padding = padding_len)
        self.conv4 = nn.Conv2d(in_channels = filters[2],out_channels = filters[3],kernel_size = filter_size[3],stride = stride_len,padding = padding_len)
        self.conv5 = nn.Conv2d(in_channels = filters[3],out_channels = filters[4],kernel_size = filter_size[4],stride = stride_len,padding = padding_len)

        self.pool = nn.MaxPool2d(kernel_size=max_pool_kernel_size,stride = max_pool_stride_len)

        self.drop = nn.Dropout2d(p=dropdout_ratio)


        self.post_conv_output_len = Out_dim_H*Out_dim_W*filters[4]
        #fully connected layer
        self.fc1 = nn.Linear(in_features = Out_dim_H*Out_dim_W*filters[4],out_features = dense_neuron_size)
        self.fc2 = nn.Linear(in_features = dense_neuron_size,out_features = num_of_classes)



    def perform_activation(self,fn,x):
        if fn == 'relu':
            return F.relu(x)
        elif fn == 'elu':
            return F.elu(x)
        elif fn == 'leaky_relu':
            return F.leaky_relu(x)
        elif fn == 'tanh':
            return F.tanh(x)
        elif fn == 'sigmoid':
            return F.sigmoid(x)

    def forward(self,x):

        x = self.perform_activation(self.activation_fn[0],self.pool(self.conv1(x)))
        x = self.perform_activation(self.activation_fn[1],self.pool(self.conv2(x)))
        x = self.perform_activation(self.activation_fn[2],self.pool(self.conv3(x)))
        x = self.perform_activation(self.activation_fn[3],self.pool(self.conv4(x)))
        x = self.perform_activation(self.activation_fn[4],self.pool(self.conv5(x)))

        x = F.dropout(self.drop(x), training=self.training)

        x = x.view(-1, self.post_conv_output_len)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
            
        return torch.log_softmax(x, dim=1)

In [53]:
def train(model,train_loader,optimizer,epoch,device):

    model.train()
    training_loss = 0
    batch_id = 0
    print("----------------------IN EPOCH {}--------------------------------".format(epoch))
    for data,target in train_loader:
        batch_id += 1
        data,target = data.to(device),target.to(device)

        output = model(data)

        loss = loss_criteria(output,target)
        training_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("avg training loss is {:.6f}".format(training_loss/batch_id))
    return training_loss


In [52]:
def test(model,test_loader,device):
    model.eval()
    test_loss = 0
    correct = 0
    batch_id = 0
    with torch.no_grad():
        for data,target in test_loader:
            batch_id += 1
            data,target = data.to(device),target.to(device)
            output = model(data)
            loss = loss_criteria(output,target)
            test_loss += loss.item()
            _,predicted = torch.max(output.data,1)
            correct += torch.sum(target==predicted).item()
    accuracy = 100*correct/ len(test_loader.dataset)
    print("avg val loss is {:.6f} and accuracy is {:.2f}%".format(test_loss/batch_id,accuracy))
    return test_loss,accuracy

    


In [57]:

model = Network(num_of_classes=10,filters = [16,32,64,128,256],filter_size = [3,3,3,3,3],activation_fn=['relu','relu','relu','relu','relu'],dropdout_ratio = 0.2,dense_neuron_size = 128).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

loss_criteria = nn.CrossEntropyLoss()
train_loader,val_loader = load_dataset(new_training_folder_path,True,50)
epochs = 10
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        test_loss = test(model, val_loader,device)
        

Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.303337
avg val loss is 2.299310 and accuracy is 12.70%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.299962
avg val loss is 2.286598 and accuracy is 11.70%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.288812
avg val loss is 2.269110 and accuracy is 14.20%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.301489
avg val loss is 2.305411 and accuracy is 8.90%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.295550
avg val loss is 2.285270 and accuracy is 13.00%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.274202
avg val loss is 2.267018 and accuracy is 14.30%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.254074
avg val loss is 2.232807 and ac

In [ ]:
defaults = dict(
  num_of_filters = 32,
  filter_size = 3,
  rate = 2
  activation_fn='relu',
  dropdout_ratio = 0.3,
  dense_neuron_size = 128,
  lr = 0.0001
  batch_size = 50,
  epochs = 10,
  data_augmentation = True
)

wandb.init(project="cs6910-assignment2",config = defaults)
config = wandb.config

In [ ]:
filters = []
filter_size = []
activation_fn = []  
rate = 1
for i in range(5):
    activation_fn.append(config.activation_fn)
    filter_size.append(config.filter_size)
    filters.append(config.num_of_filter*rate)
    rate *= config.rate

model = Network(num_of_classes=10,filters = filters,filter_size = filter_size,activation_fn=activation_fn,dropdout_ratio = config.dropout_ratio,dense_neuron_size = config.dense_neuron_size,stride_len = 1,padding_len = config.padding_len,max_pool_kernel_size = config.max_pool_kernel_size,max_pool_stride_len = config.max_pool_stride_len).to(device)
optimizer = optim.Adam(model.parameters(), lr=config.lr)

loss_criteria = nn.CrossEntropyLoss()
train_loader,val_loader = load_dataset(new_training_folder_path,config.data_augmentation,config.batch_size)
epochs = config.epochs
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        val_loss,accuracy = test(model, val_loader,device)
        wandb.log({"epoch": epoch,"accuracy" : accuracy,"training loss":train_loss,"validation loss": val_loss })

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [0.0001, 0.0005,0.0008]
        },
        'activation_fn': {
            'values': ['relu', 'tanh']
        },
        'num_of_filters': {
            'values' : [32,64]
        },
        'filter_size' : {
            'values' : [3,5]
        },
        'filter_organization':{
            'values': [2,1]
        },
        'batch_size':{
            'values': [100,200]
        },
        'dropdout_ratio': {
            'values': [0.2,0.3,0.4]
        },
        'dense_neuron_size':{
            'values' : [200,100]
        },
        'data_augmentation':{
            'values': [True,False]
        },
        'epochs':{
            'values' : [10,5,15]
        },

    }
}

In [ ]:
def experiment():
    defaults = dict(
    num_of_filters = 32,
    filter_size = 3,
    filter_organization = 2
    activation_fn='relu',
    dropdout_ratio = 0.03,
    dense_neuron_size = 20000,
    lr = 0.01
    batch_size = 50,
    epochs = 10,
    data_augmentation = True
    )

    wandb.init(project="cs6910-assignment2",config = defaults)
    config = wandb.config

    filters = []
    filter_size = []
    activation_fn = []  
    rate = 1
    for i in range(5):
        activation_fn.append(config.activation_fn)
        filter_size.append(config.filter_size)
        filters.append(config.num_of_filter*rate)
        rate *= config.filter_organization

    model = Network(num_of_classes=10,filters = filters,filter_size = filter_size,activation_fn=activation_fn,dropdout_ratio = config.dropout_ratio,dense_neuron_size = config.dense_neuron_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config.lr)

    loss_criteria = nn.CrossEntropyLoss()
    train_loader,val_loader = load_dataset(new_training_folder_path,config.data_augmentation,config.batch_size)
    epochs = config.epochs
    print('Training on', device)
    for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        val_loss,accuracy = test(model, val_loader,device)
        wandb.log({"epoch": epoch,"accuracy" : accuracy,"training loss":train_loss,"validation loss": val_loss })